# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10_24M_altbinaria_SMOTE.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(380927, 679)

In [4]:
X_train = data.drop(['clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [5]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [6]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), # segun statquest esto deberia ir de 8 a 32
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [7]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v10" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-20 23:19:03,527] Using an existing study with name 'competencia2_lgbm_v10' instead of creating a new one.


In [8]:
data['clase_peso'].value_counts()

clase_peso
1.00000    359337
1.00002     20089
1.00001      1501
Name: count, dtype: int64

In [9]:
study.optimize(objective, n_trials=100) # Habia hecho 30, hago 70 mas porque parece que es un modelo superior

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[945]	cv_agg's valid gan_eval: 9.12006e+08 + 2.06596e+06


[I 2024-11-20 23:24:19,478] Trial 1 finished with value: 4560031000.0 and parameters: {'num_leaves': 53, 'learning_rate': 0.009437642992441039, 'min_data_in_leaf': 1840, 'feature_fraction': 0.1992073070288434, 'bagging_fraction': 0.6311840217710757}. Best is trial 1 with value: 4560031000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[863]	cv_agg's valid gan_eval: 9.11145e+08 + 2.39031e+06


[I 2024-11-20 23:28:57,980] Trial 2 finished with value: 4555726000.0 and parameters: {'num_leaves': 20, 'learning_rate': 0.027836168597510303, 'min_data_in_leaf': 108, 'feature_fraction': 0.3048711490539069, 'bagging_fraction': 0.6927317087491155}. Best is trial 1 with value: 4560031000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[796]	cv_agg's valid gan_eval: 9.147e+08 + 1.89701e+06


...

[I 2024-11-21 05:43:20,052] Trial 98 finished with value: 4579785000.0 and parameters: {'num_leaves': 138, 'learning_rate': 0.022548785477682503, 'min_data_in_leaf': 1002, 'feature_fraction': 0.29405291295871483, 'bagging_fraction': 0.26167588524283497}. Best is trial 95 with value: 4589060000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[564]	cv_agg's valid gan_eval: 9.1575e+08 + 1.77819e+06


[I 2024-11-21 05:46:38,825] Trial 99 finished with value: 4578749000.0 and parameters: {'num_leaves': 144, 'learning_rate': 0.018683126702980824, 'min_data_in_leaf': 878, 'feature_fraction': 0.3511404710977656, 'bagging_fraction': 0.31237322111999183}. Best is trial 95 with value: 4589060000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[912]	cv_agg's valid gan_eval: 9.16495e+08 + 2.58136e+06


[I 2024-11-21 05:51:12,865] Trial 100 finished with value: 4582473000.0 and parameters: {'num_leaves': 142, 'learning_rate': 0.014669981618418454, 'min_data_in_leaf': 853, 'feature_fraction': 0.2748182141416816, 'bagging_fraction': 0.3921122343032548}. Best is trial 95 with value: 4589060000.0.


Analizamos los resultados as usual

In [10]:
optuna.visualization.plot_optimization_history(study)

In [11]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [12]:
plot_slice(study)

In [13]:
plot_contour(study)

In [14]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [15]:
study.best_trial.params

{'num_leaves': 142,
 'learning_rate': 0.016668881507013875,
 'min_data_in_leaf': 1002,
 'feature_fraction': 0.3071733626128024,
 'bagging_fraction': 0.2640693062857709}

In [16]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

778